In [1]:
import json
from utils import *

# Load the provided JSON raw data from the attachment
json_file_path = "./data/raw/appraisals_dataset.json"

with open(json_file_path, 'r') as file:
    raw_data = json.load(file)

In [2]:
# Initialize lists to store processed data
subjects = []
comps = []
properties = []

# Process each appraisal
for appraisal in raw_data.get('appraisals', []):
    # Process subject property
    if 'subject' in appraisal:
        subject_data = appraisal['subject'].copy()
        # Add standardized address
        subject_data.update(process_subject_address(subject_data))
        subjects.append(subject_data)
    
    # Process comp properties
    if 'comps' in appraisal:
        for comp in appraisal['comps']:
            comp_data = comp.copy()
            # Add standardized address
            comp_data.update(process_comp_address(comp_data))
            # Add reference to subject property
            if 'subject' in appraisal and 'address' in appraisal['subject']:
                comp_data['subject_address'] = appraisal['subject']['address']
            comps.append(comp_data)
    
    # Process available properties
    if 'properties' in appraisal:
        for prop in appraisal['properties']:
            prop_data = prop.copy()
            # Add standardized address
            prop_data.update(process_property_address(prop_data))
            # Add reference to subject property
            if 'subject' in appraisal and 'address' in appraisal['subject']:
                prop_data['subject_address'] = appraisal['subject']['address']
            properties.append(prop_data)

# Convert to DataFrames
subjects_df = pd.DataFrame(subjects)
comps_df = pd.DataFrame(comps)
properties_df = pd.DataFrame(properties)

subjects_df.head()
comps_df.head()
properties_df.head()

# Save processed data to CSV files for further analysis
subjects_df.to_csv("./data/processed/processed_subjects_v1.csv", index=True)
comps_df.to_csv("./data/processed/processed_comps_v1.csv", index=True)
properties_df.to_csv("./data/processed/processed_properties_v1.csv", index=True)

print("Processed data saved to CSV files.")

Processed data saved to CSV files.


In [3]:
# Examine the parsed addresses
std_address_cols = ['std_unit_number', 'std_street_number', 'std_street_name', 'std_city', 'std_province', 'std_postal_code', 'std_full_address']

subjects_address_cols = ['address', 'subject_city_province_zip'] + std_address_cols
comps_address_cols = ['address', 'city_province'] + std_address_cols
properties_address_cols = ['address', 'city', 'province'] + std_address_cols

l, r = 0, 100

subjects_df[subjects_address_cols][l:r].to_csv('./data/addresses/subjects_sample_addresses.csv', index=True)
comps_df[comps_address_cols][l:r].to_csv('./data/addresses/comps_sample_addresses.csv', index=True)
properties_df[properties_address_cols][l:r].to_csv('./data/addresses/properties_sample_addresses.csv', index=True)

print("Processed data saved to CSV files.")

print("Sample of standardized subject addresses:")
display(subjects_df[subjects_address_cols].head())

print("\nSample of standardized comp addresses:")
display(comps_df[comps_address_cols].head())

print("\nSample of standardized property addresses:")
display(properties_df[properties_address_cols].head())

Processed data saved to CSV files.
Sample of standardized subject addresses:


,address,subject_city_province_zip,std_unit_number,std_street_number,std_street_name,std_city,std_province,std_postal_code,std_full_address
0,142-950 Oakview Ave Kingston ON K7M 6W8,"""Twin Oak Meadows""",142,950,Oakview Ave Kingston On K7M 6W8,Twin Oak,MEADOWS,None,"950 Oakview Ave Kingston On K7M 6W8, Unit 142,..."
1,7180 207 HWY Halifax NS B0J2L0,"West Chezzetcook, NS B0J2L0",None,7180,207 Hwy Halifax Ns B0J2L0,"West Chezzetcook,",NS,B0J2L0,"7180 207 Hwy Halifax Ns B0J2L0, West Chezzetco..."
2,11 PAUL AVE Ayr ON N0B1E0,Ayr ON N0B1E0,None,11,Paul Ave,Ayr,ON,N0B1E0,"11 Paul Ave, Ayr, ON, N0B1E0"
3,102 Stonewalk Dr Kemptville ON K0G 1J0,Kemptville ON K0G 1J0,None,102,Stonewalk Dr,Kemptville,ON,K0G1J0,"102 Stonewalk Dr, Kemptville, ON, K0G1J0"
4,407 105 Dunbrack St Halifax NS B3M3G7,Halifax NS B3M3G7,None,407,105 Dunbrack St,Halifax,NS,B3M3G7,"407 105 Dunbrack St, Halifax, NS, B3M3G7"



Sample of standardized comp addresses:


,address,city_province,std_unit_number,std_street_number,std_street_name,std_city,std_province,std_postal_code,std_full_address
0,930 Amberdale Cres,Kingston ON K7M 6V1,None,930,Amberdale Cres,Kingston,ON,K7M6V1,"930 Amberdale Cres, Kingston, ON, K7M6V1"
1,771 Ashwood Dr,Kingston ON K7M 6X7,None,771,Ashwood Dr,Kingston,ON,K7M6X7,"771 Ashwood Dr, Kingston, ON, K7M6X7"
2,995 Amberdale Cres,Kingston ON K7M 6X6,None,995,Amberdale Cres,Kingston,ON,K7M6X6,"995 Amberdale Cres, Kingston, ON, K7M6X6"
3,64 Deermist Dr,Porters Lake NS B3E 1P3,None,64,Deermist Dr,Porters Lake,NS,B3E1P3,"64 Deermist Dr, Porters Lake, NS, B3E1P3"
4,85 Oceanic Dr,East Lawrencetown NS B2Z 1T6,None,85,Oceanic Dr,East Lawrencetown,NS,B2Z1T6,"85 Oceanic Dr, East Lawrencetown, NS, B2Z1T6"



Sample of standardized property addresses:


,address,city,province,std_unit_number,std_street_number,std_street_name,std_city,std_province,std_postal_code,std_full_address
0,463 Conservatory Dr,Kingston,Ontario,None,463,Conservatory Dr,Kingston,ON,K7M9C8,"463 Conservatory Dr, Kingston, ON, K7M9C8"
1,463 Conservatory Drive,Kingston,Ontario,None,463,Conservatory Dr,Kingston,ON,K7M9C8,"463 Conservatory Dr, Kingston, ON, K7M9C8"
2,311 Janette St,Kingston,Ontario,None,311,Janette St,Kingston,ON,K7P0K8,"311 Janette St, Kingston, ON, K7P0K8"
3,311 Janette Street,Kingston,Ontario,None,311,Janette St,Kingston,ON,K7P0K8,"311 Janette St, Kingston, ON, K7P0K8"
4,4056 Bath Rd,Kingston,Ontario,None,4056,Bath Rd,Kingston,ON,K7M4Y4,"4056 Bath Rd, Kingston, ON, K7M4Y4"


In [4]:
# Clean the DataFrames (basic cleaning)
subjects_df = clean_dataframe(subjects_df)
comps_df = clean_dataframe(comps_df)
properties_df = clean_dataframe(properties_df)

# Save processed data to CSV files for further analysis
subjects_df.to_csv("./data/processed/processed_subjects_v2.csv", index=True)
comps_df.to_csv("./data/processed/processed_comps_v2.csv", index=True)
properties_df.to_csv("./data/processed/processed_properties_v2.csv", index=True)

print("Processed data saved to CSV files.")

Processed data saved to CSV files.


In [5]:
# Apply specific processing to fields that need special handling
    
# Process sale_price in comps
if 'sale_price' in comps_df.columns:
    comps_df['sale_price'] = comps_df['sale_price'].apply(process_sale_price)

# Process bedroom counts
if 'num_beds' in subjects_df.columns:
    subjects_df['bedrooms'] = subjects_df['num_beds'].apply(process_bedroom_count)
    # Keep original column for reference if needed
    subjects_df.drop('num_beds', axis=1, inplace=True)

if 'bed_count' in comps_df.columns:
    comps_df['bedrooms'] = comps_df['bed_count'].apply(process_bedroom_count)
    # Keep original column for reference if needed
    comps_df.drop('bed_count', axis=1, inplace=True)

# Process bathroom counts
if 'num_baths' in subjects_df.columns:
    bath_results = subjects_df['num_baths'].apply(process_bathroom_count)
    subjects_df['full_baths'] = [result[0] for result in bath_results]
    subjects_df['half_baths'] = [result[1] for result in bath_results]
    # Keep original column for reference if needed
    subjects_df.drop('num_baths', axis=1, inplace=True)

if 'bath_count' in comps_df.columns:
    bath_results = comps_df['bath_count'].apply(process_bathroom_count)
    comps_df['full_baths'] = [result[0] for result in bath_results]
    comps_df['half_baths'] = [result[1] for result in bath_results]
    # Keep original column for reference if needed
    comps_df.drop('bath_count', axis=1, inplace=True)

# Process GLA (Gross Living Area)
if 'gla' in subjects_df.columns:
    subjects_df['gla'] = subjects_df['gla'].apply(process_gla)

if 'gla' in comps_df.columns:
    comps_df['gla'] = comps_df['gla'].apply(process_gla)

if 'gla' in properties_df.columns:
    properties_df['gla'] = properties_df['gla'].apply(process_gla)

# Save processed data to CSV files for further analysis
subjects_df.to_csv("./data/processed/processed_subjects_v3.csv", index=True)
comps_df.to_csv("./data/processed/processed_comps_v3.csv", index=True)
properties_df.to_csv("./data/processed/processed_properties_v3.csv", index=True)

print("Processed data saved to CSV files.")

Processed data saved to CSV files.


In [6]:
# Remove units from numeric fields
numeric_fields_with_units = [
    'distance_to_subject', 
    'lot_size',
    'lot_size_sf',
    'main_lvl_area',
    'second_lvl_area',
    'third_lvl_area',
    'basement_area',
    'main_level_finished_area',
    'upper_lvl_fin_area'
]

for df in [subjects_df, comps_df, properties_df]:
    for field in numeric_fields_with_units:
        if field in df.columns:
            df[field] = df[field].apply(remove_units_and_symbols)

# Save processed data to CSV files for further analysis
subjects_df.to_csv("./data/processed/processed_subjects_v4.csv", index=True)
comps_df.to_csv("./data/processed/processed_comps_v4.csv", index=True)
properties_df.to_csv("./data/processed/processed_properties_v4.csv", index=True)

print("Processed data saved to CSV files.")

Processed data saved to CSV files.


In [7]:
# Convert column types
mapping_df = pd.read_csv("./data/mappings/complete_field_mappings.csv")

subjects_df = convert_column_types(subjects_df, mapping_df[mapping_df['section'] == 'subject'], 'subject')
comps_df = convert_column_types(comps_df, mapping_df[mapping_df['section'] == 'comps'], 'comps')
properties_df = convert_column_types(properties_df, mapping_df[mapping_df['section'] == 'properties'], 'properties')

# Save processed data to CSV files for further analysis
subjects_df.to_csv("./data/processed/processed_subjects_v5.csv", index=True)
comps_df.to_csv("./data/processed/processed_comps_v5.csv", index=True)
properties_df.to_csv("./data/processed/processed_properties_v5.csv", index=True)

print("Processed data saved to CSV files.")

Processed data saved to CSV files.


In [8]:
print("Subject Properties:")
display(subjects_df.head())
print("Missing values in Subject Properties:")
display(subjects_df.isna().sum())

print("\nComp Properties:")
display(comps_df.head())
print("\nMissing values in Comp Properties:")
display(comps_df.isna().sum())

print("\nAvailable Properties:")
display(properties_df.head())
print("\nMissing values in Available Properties:")
display(properties_df.isna().sum())

Subject Properties:


,municipality_district,year_built,structure_type,roof,construction,remaining_economic_life,windows,basement,basement_area,foundation_walls,...,std_unit_number,std_street_number,std_street_name,std_city,std_province,std_postal_code,std_full_address,bedrooms,full_baths,half_baths
0,Kingston,1976,Townhouse,Asphalt Shingle,Wood Frame,50,Vinyl,Full/Finished,522.0,Concrete,...,142,950,Oakview Ave Kingston On K7M 6W8,Twin Oak,MEADOWS,NaN,"950 Oakview Ave Kingston On K7M 6W8, Unit 142,...",3.0,1.0,1.0
1,Halifax Regional Municipality - West Chezzetcook,2011,Detached,Asphalt Shingle,Log,50,Wood,Full/Finished,1060.0,Poured Concrete,...,NaN,7180,207 Hwy Halifax Ns B0J2L0,"West Chezzetcook,",NS,B0J2L0,"7180 207 Hwy Halifax Ns B0J2L0, West Chezzetco...",3.0,2.0,1.0
2,"Township of North Dumfries, Region of Waterloo",1983,Detached,Asphalt/Fiberglass Shingle,Wood,65,PVC,Full/Part Finished,1660.0,Concrete,...,NaN,11,Paul Ave,Ayr,ON,N0B1E0,"11 Paul Ave, Ayr, ON, N0B1E0",4.0,2.0,1.0
3,"North Grenville, Ontario",<NA>,Detached,Asphalt/Fiberglass Shingle,Wood,60,Vinyl,Full/Finished,1283.0,Poured concrete,...,NaN,102,Stonewalk Dr,Kemptville,ON,K0G1J0,"102 Stonewalk Dr, Kemptville, ON, K0G1J0",2.0,3.0,0.0
4,Halifax Regional Municipality - Halifax,1978,Condominium,Tar & Gravel,Steel,55,PVC,None,NaN,Concrete,...,NaN,407,105 Dunbrack St,Halifax,NS,B3M3G7,"407 105 Dunbrack St, Halifax, NS, B3M3G7",3.0,2.0,0.0


Missing values in Subject Properties:


municipality_district       1
year_built                  9
structure_type              1
roof                        1
construction                1
remaining_economic_life     3
windows                     1
basement                    3
basement_area              12
foundation_walls            1
flooring                    1
plumbing_lines              1
heating                     1
fuel_type                   1
cooling                     1
room_count                  1
room_total                  1
main_lvl_area              22
second_lvl_area            45
third_lvl_area             87
gla                         0
condition                   0
std_unit_number            85
std_street_number           0
std_street_name             0
std_city                    0
std_province                0
std_postal_code             1
std_full_address            0
bedrooms                    1
full_baths                  1
half_baths                  1
dtype: int64


Comp Properties:


...

distance_to_subject      5
prop_type                3
stories                  0
sale_price               0
dom                    126
location_similarity      3
condition_relative      69
gla                      0
room_count               7
basement_finish          9
parking                  0
neighborhood           261
std_unit_number        227
std_street_number        0
std_street_name          0
std_city                 0
std_province             1
std_postal_code          1
std_full_address         0
subject_address          0
bedrooms                 0
full_baths               0
half_baths               0
dtype: int64


Available Properties:


,id,bedrooms,gla,property_sub_type,structure_type,room_count,full_baths,half_baths,main_level_finished_area,upper_lvl_fin_area,...,latitude,longitude,std_unit_number,std_street_number,std_street_name,std_city,std_province,std_postal_code,std_full_address,subject_address
0,367,3,1500.0,Detached,Detached,11,3,<NA>,NaN,NaN,...,44.2325,-76.5901,NaN,463,Conservatory Dr,Kingston,ON,K7M9C8,"463 Conservatory Dr, Kingston, ON, K7M9C8",142-950 Oakview Ave Kingston ON K7M 6W8
1,163443,3,1750.0,Detached,"Detached, 2-Storey",11,<NA>,<NA>,NaN,NaN,...,44.2325,-76.5901,NaN,463,Conservatory Dr,Kingston,ON,K7M9C8,"463 Conservatory Dr, Kingston, ON, K7M9C8",142-950 Oakview Ave Kingston ON K7M 6W8
2,378,3,1500.0,Freehold Townhouse,Freehold Townhouse,11,4,<NA>,NaN,NaN,...,44.2622,-76.5904,NaN,311,Janette St,Kingston,ON,K7P0K8,"311 Janette St, Kingston, ON, K7P0K8",142-950 Oakview Ave Kingston ON K7M 6W8
3,130023,3,1300.0,Freehold Townhouse,"Freehold Townhouse, 2-Storey",11,<NA>,<NA>,NaN,NaN,...,44.2622,-76.5904,NaN,311,Janette St,Kingston,ON,K7P0K8,"311 Janette St, Kingston, ON, K7P0K8",142-950 Oakview Ave Kingston ON K7M 6W8
4,2782,4,NaN,Rural Resid,Rural Resid,13,4,0,NaN,NaN,...,44.2407,-76.6102,NaN,4056,Bath Rd,Kingston,ON,K7M4Y4,"4056 Bath Rd, Kingston, ON, K7M4Y4",142-950 Oakview Ave Kingston ON K7M 6W8



Missing values in Available Properties:


id                             0
bedrooms                     173
gla                          176
property_sub_type             70
structure_type                38
room_count                   137
full_baths                  3424
half_baths                  6272
main_level_finished_area    7064
upper_lvl_fin_area          8418
bg_fin_area                 9820
year_built                  4026
roof                        5259
basement                    3110
cooling                     3335
heating                      158
close_price                   72
public_remarks               303
latitude                       0
longitude                      0
std_unit_number             6560
std_street_number            314
std_street_name                0
std_city                       3
std_province                   0
std_postal_code               41
std_full_address               0
subject_address                0
dtype: int64

In [9]:
# Summary statistics before DEDUPLICATION
subjects_predup_len = len(subjects_df)
comps_predup_len = len(comps_df)
properties_predup_len = len(properties_df)

print("\n=== SUMMARY BEFORE DEDUPLICATION ===")
print(f"Total subject properties: {subjects_predup_len}")
print(f"Total comp properties: {comps_predup_len}")
print(f"Total available properties: {properties_predup_len}")


=== SUMMARY BEFORE DEDUPLICATION ===
Total subject properties: 88
Total comp properties: 264
Total available properties: 9820


In [10]:
# Handle duplicates in all three dataframes by keeping the most complete records.

print("=== HANDLING DUPLICATES ===")

# Define deduplication keys for each dataframe using the std_ fields
subject_dedup_keys = ['std_street_number', 'std_street_name', 'std_city', 'std_postal_code']
comps_dedup_keys = ['std_street_number', 'std_street_name', 'std_city', 'std_postal_code']
properties_dedup_keys = ['std_street_number', 'std_street_name', 'std_city', 'std_postal_code']

# Filter keys to only include columns that exist
subject_dedup_keys = [k for k in subject_dedup_keys if k in subjects_df.columns]
comps_dedup_keys = [k for k in comps_dedup_keys if k in comps_df.columns]
properties_dedup_keys = [k for k in properties_dedup_keys if k in properties_df.columns]

# Deduplicate each dataframe
print("\nDeduplicating Subject Properties:")
subjects_df = merge_duplicates_keep_most_complete(subjects_df, subject_dedup_keys)

print("\nDeduplicating Comp Properties:")
comps_df = merge_duplicates_keep_most_complete(comps_df, comps_dedup_keys)

print("\nDeduplicating Available Properties:")
properties_df = merge_duplicates_keep_most_complete(properties_df, properties_dedup_keys)

# Summary statistics after DEDUPLICATION
subjects_postdup_len = len(subjects_df)
comps_postdup_len = len(comps_df)
properties_postdup_len = len(properties_df)

print("\n=== SUMMARY AFTER DEDUPLICATION ===")
print(f"Total subject properties: {subjects_postdup_len} (Removed {subjects_predup_len - subjects_postdup_len})")
print(f"Total comp properties: {comps_postdup_len} (Removed {comps_predup_len - comps_postdup_len})")
print(f"Total available properties: {properties_postdup_len} (Removed {properties_predup_len - properties_postdup_len})")

=== HANDLING DUPLICATES ===

Deduplicating Subject Properties:
Number of entries before merging duplicates: 88
Number of entries after merging duplicates: 88
Removed 0 duplicate entries

Deduplicating Comp Properties:
Number of entries before merging duplicates: 264
Number of entries after merging duplicates: 255
Removed 9 duplicate entries

Deduplicating Available Properties:
Number of entries before merging duplicates: 9820
Number of entries after merging duplicates: 6028
Removed 3792 duplicate entries

=== SUMMARY AFTER DEDUPLICATION ===
Total subject properties: 88 (Removed 0)
Total comp properties: 255 (Removed 9)
Total available properties: 6028 (Removed 3792)


In [11]:
# Save processed data to CSV files for further analysis
subjects_df.to_csv("./data/processed/processed_subjects.csv", index=True)
comps_df.to_csv("./data/processed/processed_comps.csv", index=True)
properties_df.to_csv("./data/processed/processed_properties.csv", index=True)

print("Processed data saved to CSV files.")

Processed data saved to CSV files.
